In [3]:
from PIL import Image, ImageDraw
import face_recognition
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearnex.model_selection import train_test_split
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential, load_model
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
import numpy as np
from time import time

In [4]:
def eye_cropper(image_path):
    frame = cv2.imread(image_path)  # Read the image using OpenCV
    if frame is None:
        return None
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    facial_features_list = face_recognition.face_landmarks(gray)

    try:
        eye = facial_features_list[0]['left_eye']
    except:
        try:
            eye = facial_features_list[0]['right_eye']
        except:
            return None

    x_max = max([coordinate[0] for coordinate in eye])
    x_min = min([coordinate[0] for coordinate in eye])
    y_max = max([coordinate[1] for coordinate in eye])
    y_min = min([coordinate[1] for coordinate in eye])

    x_range = x_max - x_min
    y_range = y_max - y_min

    if x_range > y_range:
        right = round(.5 * x_range) + x_max
        left = x_min - round(.5 * x_range)
        bottom = round((((right - left) - y_range)) / 2) + y_max
        top = y_min - round((((right - left) - y_range)) / 2)
    else:
        bottom = round(.5 * y_range) + y_max
        top = y_min - round(.5 * y_range)
        right = round((((bottom - top) - x_range)) / 2) + x_max
        left = x_min - round((((bottom - top) - x_range)) / 2)

    cropped = frame[top:(bottom + 1), left:(right + 1)]

    cropped = cv2.resize(cropped, (80, 80))
    image_for_prediction = cropped.reshape(-1, 80, 80, 3)

    return image_for_prediction


In [5]:
def load_images_from_folder(folder, eyes = 0):
    count = 0
    error_count = 0
    images = []
    for filename in os.listdir(folder):
        try:
            img = cv2.imread(os.path.join(folder,filename))
            img = cv2.resize(img, (80,80)) ## Resizing the images
            ## for eyes if it is 0: open, 1: close
            images.append([img, eyes])
        except:
            error_count += 1
            print('ErrorCount = ' + str(error_count))
            continue
        
        count += 1
        if count % 500 == 0:
            print('Succesful Image Import Count = ' + str(count))

    return images

folder="./dataset/Open_Eyes"
open_eyes = load_images_from_folder(folder, 0)

folder="./dataset/Closed_Eyes"
closed_eyes = load_images_from_folder(folder, 1)
eyes = closed_eyes + open_eyes

Succesful Image Import Count = 500
Succesful Image Import Count = 1000
Succesful Image Import Count = 1500
Succesful Image Import Count = 2000
Succesful Image Import Count = 500
Succesful Image Import Count = 1000
Succesful Image Import Count = 1500
Succesful Image Import Count = 2000


In [6]:
X = [] 
y = [] 
for features, label in eyes: 
     X.append(features)
     y.append(label)

In [7]:
X = np.array(X).reshape(-1, 80, 80, 3)
y = np.array(y)
X = X/255.0

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)


In [9]:
model = Sequential()

# Adding first three convolutional layers
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu', # activation function 
                input_shape = (80,80,3) # shape of input (image)
                ))
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))

# Adding pooling after convolutional layers
model.add(MaxPooling2D(pool_size = (2,2))) # Dimensions of the region that you are pooling

# Adding second set of convolutional layers
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))
model.add(Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ))

# Add last pooling layer.
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Adding first dense layer with 256 nodes
model.add(Dense(256, activation='relu'))

# Adding a dropout layer to avoid overfitting
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3)) 

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# adding output layer
model.add(Dense(1, activation = 'sigmoid'))

# compiling the model
model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=[tf.keras.metrics.AUC(curve = 'PR')])

start = time()
# fitting the model
model.fit(X_train,
            y_train,
            batch_size=32,
            validation_data=(X_test, y_test),
            epochs=24)
end = time()

# evaluate the model 
model.evaluate(X_test, y_test, verbose=1)

Epoch 1/24
94/94 [==============================] - 12s 110ms/step - loss: 0.3843 - auc: 0.8985 - val_loss: 0.1402 - val_auc: 0.9846
Epoch 2/24
94/94 [==============================] - 10s 105ms/step - loss: 0.1122 - auc: 0.9874 - val_loss: 0.0736 - val_auc: 0.9963
Epoch 3/24
94/94 [==============================] - 10s 107ms/step - loss: 0.0658 - auc: 0.9962 - val_loss: 0.0315 - val_auc: 0.9986
Epoch 4/24
94/94 [==============================] - 10s 107ms/step - loss: 0.0776 - auc: 0.9940 - val_loss: 0.0570 - val_auc: 0.9989
Epoch 5/24
94/94 [==============================] - 10s 107ms/step - loss: 0.0448 - auc: 0.9971 - val_loss: 0.0686 - val_auc: 0.9896
Epoch 6/24
94/94 [==============================] - 10s 108ms/step - loss: 0.0417 - auc: 0.9986 - val_loss: 0.0394 - val_auc: 0.9961
Epoch 7/24
94/94 [==============================] - 10s 108ms/step - loss: 0.0160 - auc: 0.9991 - val_loss: 0.0208 - val_auc: 0.9972
Epoch 8/24
94/94 [==============================] - 10s 110ms/step - 

[0.007898536510765553, 0.9999880194664001]

In [10]:
model.save('./dataset/drowsiness_detection.h5')

In [11]:
def model_responce(image, model):    
    image_for_prediction = eye_cropper(image)
    if image_for_prediction is None:
        return 'Yes'
    try:
        image_for_prediction = image_for_prediction/255.0
    except:
        print("Error")
    
    prediction = model.predict(image_for_prediction)
    print(prediction)
    # Based on prediction, display either "Open Eyes" or "Closed Eyes"
    if prediction < 0.7:
        status = 'No'
    else:
        status = 'Yes'
    return status

In [ ]:
_ = model_responce(image, model) #input image here
print(_)